In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

data = pd.read_csv("D:/kuliah/smt-5/pertumbuhan_ekonomi_indo.csv")
data.head()

,Tahun,Inflasi,Pertumbuhan Ekonomi,Jumlah Uang Beredar/M2,Ekspor,Impor
0,1990,9.53,7.24,84.63,25675.3,21837.0
1,1991,9.52,6.95,99341.00,29142.4,25868.8
2,1992,4.94,6.45,119053.00,33967.0,27279.6
3,1993,9.77,6.49,145202.00,36823.0,28327.8
4,1994,9.24,7.49,174512.00,40053.4,31983.5


In [17]:
# Mendefinisikan fungsi ADF test
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    return result[1] 

# Fungsi untuk ADF Test
def adf_test(series, column_name):
    result = adfuller(series, autolag='AIC')
    print(f"ADF Test for {column_name}:")
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("-" * 30)
    return result[1]

# Melakukan ADF test untuk semua variabel
variables = ['Inflasi', 'Pertumbuhan Ekonomi','Jumlah Uang Beredar/M2', 
             'Ekspor ', 'Impor']

for var in variables:
    adf_test(data[var], var)

ADF Test for Inflasi:
ADF Statistic: -3.3465990749721484
p-value: 0.012919722896727357
------------------------------
ADF Test for Pertumbuhan Ekonomi:
ADF Statistic: -4.1555774965124765
p-value: 0.0007827549198617825
------------------------------
ADF Test for Jumlah Uang Beredar/M2:
ADF Statistic: 4.470691265280071
p-value: 1.0
------------------------------
ADF Test for Ekspor :
ADF Statistic: 2.810172058278969
p-value: 1.0
------------------------------
ADF Test for Impor:
ADF Statistic: -0.028415414498082325
p-value: 0.9561181097714511
------------------------------


In [4]:
# Mendefinisikan fungsi ADF test
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    return result[1] 

# Menguji stasioneritas untuk Inflasi (Y) dan Pertumbuhan Ekonomi (X)
print("ADF Test for Inflasi (Y):")
adf_test(data['Inflasi'])

print("\nADF Test for Pertumbuhan Ekonomi (X):")
adf_test(data['Pertumbuhan Ekonomi'])

ADF Test for Inflasi (Y):
ADF Statistic: -3.3465990749721484
p-value: 0.012919722896727357

ADF Test for Pertumbuhan Ekonomi (X):
ADF Statistic: -4.1555774965124765
p-value: 0.0007827549198617825


np.float64(0.0007827549198617825)

In [5]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import pandas as pd

# Contoh Data
data = data[['Inflasi', 'Pertumbuhan Ekonomi']]

# 1. Membuat model jangka panjang (long-term model)
long_term_model = sm.OLS(data['Inflasi'], sm.add_constant(data['Pertumbuhan Ekonomi'])).fit()

# 2. Menyimpan residual sebagai Error Correction Term (ECT)
data['ECT'] = long_term_model.resid
print(long_term_model.summary())

# 3. Uji stasioneritas pada residual (ECT) menggunakan ADF Test
adf_result = adfuller(data['ECT'])

# 4. Menampilkan hasil uji ADF
print(f'\nADF Statistic: {adf_result[0]}')
print('p-value:', adf_result[1])
print('Critical Values:', adf_result[4])

# 5. Interpretasi hasil
if adf_result[1] < 0.05:  # Biasanya tingkat signifikansi 5%
    print("Residual stasioner. Variabel memiliki hubungan kointegrasi.")
else:
    print("Residual tidak stasioner. Variabel tidak memiliki hubungan kointegrasi.")

                            OLS Regression Results                            
Dep. Variable:                Inflasi   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     45.35
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           1.55e-07
Time:                        19:09:02   Log-Likelihood:                -115.71
No. Observations:                  33   AIC:                             235.4
Df Residuals:                      31   BIC:                             238.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  21.4690    

In [6]:
# Step 1: Differencing
data['delta_Y'] = data['Inflasi']
data['delta_X'] = data['Pertumbuhan Ekonomi']

# Step 2: Lagging ECT
data['ECT_lag'] = data['ECT'].shift(1)

# Step 3: Drop NaN
data.dropna(inplace=True)

# Step 4: Model ECM
ecm_model = sm.OLS(data['delta_Y'], sm.add_constant(data[['delta_X', 'ECT_lag']])).fit()

# Step 5: Menampilkan Hasil
print(ecm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                delta_Y   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     23.01
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           1.03e-06
Time:                        19:09:02   Log-Likelihood:                -111.91
No. Observations:                  32   AIC:                             229.8
Df Residuals:                      29   BIC:                             234.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.4706      2.386      8.998      0.0

In [7]:
# Step 1: Differencing
data['delta_Y'] = data['Inflasi'].diff()
data['delta_X'] = data['Pertumbuhan Ekonomi'].diff()

# Step 2: Lagging ECT
data['ECT_lag'] = data['ECT'].shift(1)

# Step 3: Drop NaN
data.dropna(inplace=True)

# Step 4: Model ECM
ecm_model = sm.OLS(data['delta_Y'], sm.add_constant(data[['delta_X', 'ECT_lag']])).fit()

# Step 5: Menampilkan Hasil
print(ecm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                delta_Y   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     76.36
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           4.59e-12
Time:                        19:09:02   Log-Likelihood:                -105.69
No. Observations:                  31   AIC:                             217.4
Df Residuals:                      28   BIC:                             221.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4597      1.383     -0.332      0.7